In [2]:
import numpy as np
import pandas as pd

In [3]:
data1 = pd.read_csv(r"D:\Y4 Research\datasets\DSLD-full-database-CSV\DSLD-full-database-CSV\DietarySupplementFacts_1.csv",
                 encoding="latin1")

In [4]:
data1

,URL,DSLD ID,Product Name,Serving Size,Ingredient,DSLD Ingredient Categories,Amount Per Serving,Amount Per Serving Unit,% Daily Value per Serving,Daily Value Target Group
0,https://dsld.od.nih.gov/label/542,542,B-2 100 mg,1 Tablet(s),Riboflavin,vitamin,100,mg,5882,Adults and children 4 or more years of age
1,https://dsld.od.nih.gov/label/543,543,B-6 100 mg,1 Tablet(s),Vitamin B6,vitamin,100,mg,5000,Adults and children 4 or more years of age
2,https://dsld.od.nih.gov/label/544,544,C-1000 mg With Protective Bioflavonoids And Wi...,1 Caplet(s),Vitamin C,vitamin,1000,mg,1667,Adults and children 4 or more years of age
3,https://dsld.od.nih.gov/label/544,544,C-1000 mg With Protective Bioflavonoids And Wi...,1 Caplet(s),Citrus Bioflavonoids,non-nutrient/non-botanical,25,mg,NaN,Adults and children 4 or more years of age
4,https://dsld.od.nih.gov/label/544,544,C-1000 mg With Protective Bioflavonoids And Wi...,1 Caplet(s),Rose Hips,botanical,20,mg,NaN,Adults and children 4 or more years of age
...,...,...,...,...,...,...,...,...,...,...
301874,https://dsld.od.nih.gov/label/31483,31483,Assault Fruit Punch,14.5 Gram(s),Huperzine A,non-nutrient/non-botanical,NaN,NaN,NaN,NaN
301875,https://dsld.od.nih.gov/label/31483,31483,Assault Fruit Punch,14.5 Gram(s),Hydration System,blend,1200,mg,NaN,Adults and children 4 or more years of age
301876,https://dsld.od.nih.gov/label/31483,31483,Assault Fruit Punch,14.5 Gram(s),Taurine,non-nutrient/non-botanical,NaN,NaN,NaN,NaN
301877,https://dsld.od.nih.gov/label/31483,31483,Assault Fruit Punch,14.5 Gram(s),Coconut Water Powder,botanical,NaN,NaN,NaN,NaN


Exploring Data Set

In [5]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301879 entries, 0 to 301878
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   URL                         301879 non-null  object
 1   DSLD ID                     301879 non-null  int64 
 2   Product Name                301879 non-null  object
 3   Serving Size                301879 non-null  object
 4   Ingredient                  301879 non-null  object
 5   DSLD Ingredient Categories  301879 non-null  object
 6   Amount Per Serving          235162 non-null  object
 7   Amount Per Serving Unit     241366 non-null  object
 8   % Daily Value per Serving   124364 non-null  object
 9   Daily Value Target Group    228974 non-null  object
dtypes: int64(1), object(9)
memory usage: 23.0+ MB


In [6]:
data1.isnull().sum()

URL                                0
DSLD ID                            0
Product Name                       0
Serving Size                       0
Ingredient                         0
DSLD Ingredient Categories         0
Amount Per Serving             66717
Amount Per Serving Unit        60513
% Daily Value per Serving     177515
Daily Value Target Group       72905
dtype: int64